# Prétraitement des données

### Import

In [1]:
import pandas as P
from collections import defaultdict

### Chargement des données

In [2]:
PATH = '' # PATH = '.../data/raw/'
road_incident = P.read_csv(PATH+'roadIncident_raw.csv')
population = P.read_excel(PATH+'population.xlsx')
pib = P.read_excel(PATH+'pib.xlsx')

#### Road Incident

Renommer les colonnes

In [3]:
def rename_columns(c):
    try: 
        return {
            'name': 'pays',
            'ratedall': 'all',
            'rated_0_14': '00-14',
            'rated_15_29': '15-29',
            'rated_30_44': '30-44',
            'rated_45_59': '45-59',
            'rated_60+': '60+'
        }[c]
    except:
        return c

del road_incident['country']
road_incident.columns = [rename_columns(c) for c in road_incident.columns]

#### Population

 Renommer les colonnes


In [4]:
Tous

NameError: name 'Tous' is not defined

Suppression des colonnes inutiles

In [ ]:
del population['série Code']

annees = range(1970, 2009)


#### PIB

 Renommer les colonnes


In [ ]:
def rename_columns(c):
    if c.startswith('1') or c.startswith('2'):
        return c[:4]
    try: 
        return {
            'Country Name': 'pays',
            'Country Code': 'code'
        }[c]
    except:
        return c

pib.columns = [rename_columns(c) for c in pib.columns]

Suppression des colonnes inutiles

In [ ]:
del pib['Series Code']
del pib['Series Name']

In [ ]:
pib

### Preprocess

#### Road Incident

Déplacement de l'attribut sex de ligne vers colonnes

In [ ]:
country_incident = {'SUM':{}}
incident_columns = ['all', '00-14', '15-29', '30-44', '45-59', '60+', 'M', 'M 00-14', 'M 15-29', 'M 30-44', 'M 45-59', 'M 60+', 'F', 'F 00-14', 'F 15-29', 'F 30-44', 'F 45-59', 'F 60+']

for i, p in road_incident.iterrows():
    pays ={
     'Germany, Former Democratic Republic': 'Germany',
     'Germany, Former Federal Republic': 'Germany',
     'USSR, Former': 'Russia',
     'Russian Federation': 'Russia',
     'Republic of Moldova': 'Moldova',
     'Republic of Korea': 'South Korea',
     'United kingdom': 'United Kingdom',
     'Venezuela (Bolivarian Republic of)': 'Venezuela',
    }.get(p['pays'], p['pays'])
    if pays not in country_incident:
        country_incident[pays] = {}
    
    def store(country_name):
        c = country_incident[country_name]
        if p['year'] not in c:
            c[p['year']] = {_: 0 for _ in incident_columns}
        c = c[p['year']]
        if p['sex'] == 0:
            c['all'] += p['all']
            c['00-14'] += p['00-14']
            c['15-29'] += p['15-29']
            c['30-44'] += p['30-44']
            c['45-59'] += p['45-59']
            c['60+'] += p['60+']
        elif p['sex'] == 1:
            c['M'] += p['all']
            c['M 00-14'] += p['00-14']
            c['M 15-29'] += p['15-29']
            c['M 30-44'] += p['30-44']
            c['M 45-59'] += p['45-59']
            c['M 60+'] += p['60+']
        elif p['sex'] == 2:
            c['F'] += p['all']
            c['F 00-14'] += p['00-14']
            c['F 15-29'] += p['15-29']
            c['F 30-44'] += p['30-44']
            c['F 45-59'] += p['45-59']
            c['F 60+'] += p['60+']
    
    store(pays)
    store('SUM')

In [ ]:
def generate_data():
    for pays, pays_d in country_incident.items():
        for annee in annees:
            if annee in pays_d:
                yield (pays, annee) + tuple(pays_d[annee][c] for c in incident_columns)
            else:
                yield (pays, annee) + (float('nan'),)*len(incident_columns)

incident_df = P.DataFrame(data=list(generate_data()), columns=['pays', 'annee']+incident_columns)

In [ ]:
incident_df.sort_values(by=['pays', 'annee'], inplace=True)

#### Population

Extraction de chaque pays (les noms des colonnes sont simplifiés)

In [ ]:
country_population = {}
country_code = {}

for i, p in population.iterrows():
    pays ={
        'Egypt, Arab Rep.': 'Egypt',
        'Kyrgyz Republic': 'Kyrgyzstan',
        'Korea, Rep.': 'South Korea',
        'Slovak Republic': 'Slovakia',
        'United States': 'United States of America',
        'Venezuela, RB': 'Venezuela',
        'Russian Federation': 'Russia',
    }.get(p['pays'], p['pays'])
    
    if pays not in country_population:
        country_population[pays] = {}
        country_code[pays] = p['code']
    
    serie = {
        'Population, total': 'total',
        'Population, male': 'male',
        'Population, female': 'female',
        'Population ages 00-14, female (% of total)': 'F 00-14',
        'Population ages 00-14, male (% of total)': 'M 00-14',
        'Population ages 15-19, female (% of female population)': 'F 15-19',
        'Population ages 15-19, male (% of male population)': 'M 15-19',
        'Population ages 20-24, female (% of female population)': 'F 20-24',
        'Population ages 20-24, male (% of male population)': 'M 20-24',
        'Population ages 25-29, female (% of female population)': 'F 25-29',
        'Population ages 25-29, male (% of male population)': 'M 25-29',
        'Population ages 30-34, female (% of female population)': 'F 30-34',
        'Population ages 30-34, male (% of male population)': 'M 30-34',
        'Population ages 35-39, female (% of female population)': 'F 35-39',
        'Population ages 35-39, male (% of male population)': 'M 35-39',
        'Population ages 40-44, female (% of female population)': 'F 40-44',
        'Population ages 40-44, male (% of male population)': 'M 40-44',
        'Population ages 45-49, female (% of female population)': 'F 45-49',
        'Population ages 45-49, male (% of male population)': 'M 45-49',
        'Population ages 50-54, female (% of female population)': 'F 50-54',
        'Population ages 50-54, male (% of male population)': 'M 50-54',
        'Population ages 55-59, female (% of female population)': 'F 55-59',
        'Population ages 55-59, male (% of male population)': 'M 55-59',
    }[p['serie']]
        
    country_population[pays][serie] = p[[str(annee) for annee in annees]]

Calcul du total de population pour chaque tranche d'age qui nous interesse

In [ ]:
country_population_total = {}
field_columns = ['all', 'M', 'F', 'F 00-14', 'M 00-14', 'F 15-29', 'M 15-29', 'F 30-44', 'M 30-44', 'F 45-59', 'M 45-59', 'F 60+', 'M 60+', '00-14', '15-29', '30-44', '45-59', '60+']

country_population_total['SUM'] = {_: 0 for _ in field_columns}

for pays, p_dict in country_population.items():
    m = p_dict['male']
    f = p_dict['female']
    
    try:
        def store(country_name):
            if pays not in country_population_total:
                d = {}
                d['all'] = p_dict['total'] 
                d['M'] =  m 
                d['F'] =  f
                d['F 00-14'] = p_dict['F 00-14'] * f // 100
                d['M 00-14'] = p_dict['M 00-14'] * m // 100
                d['F 15-29'] = (p_dict['F 15-19']+p_dict['F 20-24']+p_dict['F 25-29']) * f // 100
                d['M 15-29'] = (p_dict['M 15-19']+p_dict['M 20-24']+p_dict['M 25-29']) * m // 100
                d['F 30-44'] = (p_dict['F 30-34']+p_dict['F 35-39']+p_dict['F 40-44']) * f // 100
                d['M 30-44'] = (p_dict['M 30-34']+p_dict['M 35-39']+p_dict['M 40-44']) * m // 100
                d['F 45-59'] = (p_dict['F 45-49']+p_dict['F 50-54']+p_dict['F 55-59']) * f // 100
                d['M 45-59'] = (p_dict['M 45-49']+p_dict['M 50-54']+p_dict['M 55-59']) * m // 100

                d['F 60+'] = d['F'] - d['F 00-14'] - d['F 15-29'] - d['F 30-44'] - d['F 45-59']
                d['M 60+'] = d['M'] - d['M 00-14'] - d['M 15-29'] - d['M 30-44'] - d['M 45-59']

                d['00-14'] = d['F 00-14'] + d['M 00-14']
                d['15-29'] = d['F 15-29'] + d['M 15-29']
                d['30-44'] = d['F 30-44'] + d['M 30-44']
                d['45-59'] = d['F 45-59'] + d['M 45-59']
                d['60+'] = d['F 60+'] + d['M 60+']
                
                country_population_total[country_name] = d
            else:
                d = country_population_total[country_name]

                d['all'] += p_dict['total'] 
                d['M'] +=  m 
                d['F'] +=  f
                d['F 00-14'] += p_dict['F 00-14'] * f // 100
                d['M 00-14'] += p_dict['M 00-14'] * m // 100
                d['F 15-29'] += (p_dict['F 15-19']+p_dict['F 20-24']+p_dict['F 25-29']) * f // 100
                d['M 15-29'] += (p_dict['M 15-19']+p_dict['M 20-24']+p_dict['M 25-29']) * m // 100
                d['F 30-44'] += (p_dict['F 30-34']+p_dict['F 35-39']+p_dict['F 40-44']) * f // 100
                d['M 30-44'] += (p_dict['M 30-34']+p_dict['M 35-39']+p_dict['M 40-44']) * m // 100
                d['F 45-59'] += (p_dict['F 45-49']+p_dict['F 50-54']+p_dict['F 55-59']) * f // 100
                d['M 45-59'] += (p_dict['M 45-49']+p_dict['M 50-54']+p_dict['M 55-59']) * m // 100

                d['F 60+'] += d['F'] - d['F 00-14'] - d['F 15-29'] - d['F 30-44'] - d['F 45-59']
                d['M 60+'] += d['M'] - d['M 00-14'] - d['M 15-29'] - d['M 30-44'] - d['M 45-59']

                d['00-14'] += d['F 00-14'] + d['M 00-14']
                d['15-29'] += d['F 15-29'] + d['M 15-29']
                d['30-44'] += d['F 30-44'] + d['M 30-44']
                d['45-59'] += d['F 45-59'] + d['M 45-59']
                d['60+'] += d['F 60+'] + d['M 60+']
        store('SUM')
        store(pays)
        
    except:
        print('%s ignored' % pays)

Création d'un dataframe pandas

In [ ]:
population_columns = ['pays', 'annee'] + field_columns
def generate_data():
    for pays, pays_d in country_population_total.items():
        d = {'pays': pays}
        for annee in annees:
            d['annee'] = annee
            for f in field_columns:
                d[f] = pays_d[f][str(annee)]
            yield tuple(d[_] for _ in population_columns)

population_df = P.DataFrame(data=list(generate_data()), columns=population_columns)


In [ ]:
population_df.sort_values(by=['pays', 'annee'], inplace=True)
country_info_df = P.DataFrame(data=list(country_code.items()), columns=['pays', 'code']).sort_values(by=['pays'])

#### PIB

Extraction de chaque pays (les noms des colonnes sont simplifiés)

In [ ]:
country_pib = {}

for i, p in pib.iterrows():
    pays ={
        'Egypt, Arab Rep.': 'Egypt',
        'Kyrgyz Republic': 'Kyrgyzstan',
        'Korea, Rep.': 'South Korea',
        'Slovak Republic': 'Slovakia',
        'United States': 'United States of America',
        'Venezuela, RB': 'Venezuela',
        'Russian Federation': 'Russia',
    }.get(p['pays'], p['pays'])
        
    country_pib[pays] = {_: float(p[str(_)]) if p[str(_)]!='..'  else float('nan') for _ in annees}

Création d'un dataframe pandas

In [ ]:
pib_columns = ['pays', 'annee', 'PIB']
def generate_data():
    for pays, pays_d in country_pib.items():
        d = {'pays': pays}
        for annee in annees:
            d['annee'] = annee
            d['PIB'] = pays_d[annee]
            yield tuple(d[_] for _ in pib_columns)

pib_df = P.DataFrame(data=list(generate_data()), columns=pib_columns)


In [ ]:
pib_df.sort_values(by=['pays', 'annee'], inplace=True)

In [ ]:
pib_df[pib_df.pays.isin(['United States of America'])]

#### Join datasets

In [ ]:
pays_population = set(country_info_df['pays'].unique())
pays_incident = set(incident_df['pays'].unique())
pays_pib = set(pib_df['pays'].unique())

pays_join = list(pays_population & pays_incident)
print('Ignored by join: ', pays_incident - pays_population)

#### Country Info

In [ ]:
country_info_df['Continent'] = ''

for i, c in country_info_df.iterrows():
    n = c['pays']
    if n in ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Paraguay', 'Uruguay']:
        c['Continent'] = 'Amérique du Sud'
        
    elif n in ['Canada', 'United States of America']:
        c['Continent'] = 'Amérique du Nord'
        
    elif n in ['Mexico', 'Costa Rica', 'Cuba', 'El Salvador', 'Guatemala', 'Panama', 'Trinidad and Tobago', 'Venezuela']:
        c['Continent'] = 'Amérique Centrale'
        
    elif n in ['Australia', 'New Zealand', 'Philippines']:
        c['Continent'] = 'Océanie'
        
    elif n in ['Russia', 'Belarus', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czech Republic', 'Estonia', 'Finland', 'Hungary', 'Latvia', 'Lithuania', 'Moldova','Poland','Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Ukraine']:
        c['Continent'] = 'Europe Est'
        
    elif n in ['Austria', 'Belgium', 'Cyprus', 'Denmark', 'France', 'Germany', 'Greece', 'Netherlands', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom']:
        c['Continent'] = 'Europe Ouest'
        
    elif n in ['Bahrain', 'Georgia', 'Israel', 'Kuwait']:
        c['Continent'] = 'Moyen Orient'
        
    elif n in ['Azerbaijan', 'Japan', 'Kazakhstan', 'Kyrgyzstan', 'Singapore', 'South Korea', 'Sri Lanka', 'Turkmenistan', 'Uzbekistan']:
        c['Continent'] = 'Asie'
        
    elif n in ['Egypt', 'Mauritius']:
        c['Continent'] = 'Afrique'

### Saving

In [ ]:
incident_df[incident_df.pays.isin(pays_join)].to_csv(PATH+'../incident.csv', index=False)
population_df[population_df.pays.isin(pays_join)].to_csv(PATH+'../population.csv', index=False)
pib_df[pib_df.pays.isin(pays_join)].to_csv(PATH+'../pib.csv', index=False)
country_info_df[country_info_df.pays.isin(pays_join)].to_csv(PATH+'../countryInfos.csv', index=False)